In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures


In [18]:
df_face = pd.read_csv('/kaggle/input/final-year-project/Face.csv')
df_neigh_face= pd.read_csv('/kaggle/input/final-year-project/NeighFace.csv')
df_neigh_radius= pd.read_csv('/kaggle/input/final-year-project/NeighRadius.csv')
df_radius = pd.read_csv('/kaggle/input/final-year-project/Radius.csv')

In [19]:
df_face

,Unnamed: 0,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,t1873,t1874,t1875,t1876,t1877,t1878,t1879,t1880,t1881,t1882
0,g1,16,16,16,16,16,16,16,16,16,...,0,0,0,0,0,0,0,0,0,0
1,g2,28,27,27,27,25,25,25,25,25,...,21,21,21,21,21,21,21,21,21,21
2,g3,16,16,16,16,16,16,16,16,16,...,0,0,0,0,0,0,0,0,0,0
3,g4,13,13,13,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
4,g5,12,12,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,g4996,18,18,18,18,18,18,18,18,18,...,0,0,0,0,0,0,0,0,0,0
4996,g4997,26,26,26,26,26,26,26,26,26,...,16,16,16,16,16,16,16,16,16,16
4997,g4998,14,12,12,13,13,14,13,13,13,...,0,0,0,0,0,0,0,0,0,0
4998,g4999,20,20,20,20,20,20,20,20,20,...,0,0,0,0,0,0,0,0,0,0


In [20]:
columns_to_keep = ['Unnamed: 0'] + [f't{i}' for i in range(21, 51)]

In [21]:
df_face_filtered = df_face[columns_to_keep]
df_neigh_face_filtered = df_neigh_face[columns_to_keep]
df_neigh_radius_filtered = df_neigh_radius[columns_to_keep]
df_radius_filtered = df_radius[columns_to_keep]

In [22]:
df_face_filtered

,Unnamed: 0,t21,t22,t23,t24,t25,t26,t27,t28,t29,...,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50
0,g1,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
1,g2,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
2,g3,17,17,16,15,15,15,15,15,15,...,15,15,15,15,14,14,14,14,14,14
3,g4,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
4,g5,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,g4996,18,18,18,18,18,18,18,18,17,...,17,17,17,17,17,17,17,17,17,17
4996,g4997,26,26,26,26,26,25,25,25,25,...,25,25,25,25,24,24,24,24,24,24
4997,g4998,8,8,8,9,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
4998,g4999,20,20,21,21,21,21,21,20,20,...,19,19,19,19,19,19,19,19,19,19


In [28]:
# Initialize lists to store metrics
avg_rmse_train = []
avg_rmse_test = []
avg_r2_train = []
avg_r2_test = []
equations = []

In [24]:
timeframes = [f't{i}' for i in range(21, 51)]

Models that give direct equations:
* Linear Regression
* Polynomial Regression
* Ridge and Lasso Regression
* Generalized Linear Models (GLM)
* Elastic Net Regression

**Elastic Net Regression**(Most advanced one)

In [29]:
for timeframe in timeframes:
    # Merging dataframes as per the specific timeframe
    df_merged = df_radius_filtered[['Unnamed: 0', timeframe]].rename(columns={timeframe: 'Radius'})
    df_merged = df_merged.merge(df_face_filtered[['Unnamed: 0', timeframe]], on='Unnamed: 0').rename(columns={timeframe: 'Face'})
    df_merged = df_merged.merge(df_neigh_radius_filtered[['Unnamed: 0', timeframe]], on='Unnamed: 0').rename(columns={timeframe: 'NeighRadius'})
    df_merged = df_merged.merge(df_neigh_face_filtered[['Unnamed: 0', timeframe]], on='Unnamed: 0').rename(columns={timeframe: 'NeighFace'})

    X = df_merged[['Face', 'NeighRadius', 'NeighFace']]
    y = df_merged['Radius']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Define a pipeline with StandardScaler, PolynomialFeatures, and ElasticNet
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=2)),
        ('elasticnet', ElasticNet(random_state=42))
    ])

    # Parameter grid for GridSearchCV
    param_grid = {
        'elasticnet__alpha': [0.1, 1, 10],
        'elasticnet__l1_ratio': [0.1, 0.5, 0.9]
    }

    # GridSearchCV to find the best parameters
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='r2', verbose=1)
    grid_search.fit(X_train, y_train)

    # Best model
    best_model = grid_search.best_estimator_

    # Make predictions
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Calculate and store metrics
    avg_rmse_train.append(np.sqrt(mean_squared_error(y_train, y_train_pred)))
    avg_rmse_test.append(np.sqrt(mean_squared_error(y_test, y_test_pred)))
    avg_r2_train.append(r2_score(y_train, y_train_pred))
    avg_r2_test.append(r2_score(y_test, y_test_pred))

    # Store the equation coefficients
    coef = best_model.named_steps['elasticnet'].coef_
    intercept = best_model.named_steps['elasticnet'].intercept_
    equation = f"Radius = {intercept:.4f} + {coef[1]:.4f}*Face + {coef[2]:.4f}*NeighRadius + {coef[3]:.4f}*NeighFace"
    equations.append(equation)
   

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates

In [30]:
# Output average metrics and equations
print(f"Average RMSE Train: {np.mean(avg_rmse_train)}")
print(f"Average RMSE Test: {np.mean(avg_rmse_test)}")
print(f"Average R² Train: {np.mean(avg_r2_train)}")
print(f"Average R² Test: {np.mean(avg_r2_test)}")
for eq in equations:
    print(eq)

Average RMSE Train: 0.7713212268660528
Average RMSE Test: 0.7622984041092311
Average R² Train: 0.8471550160916507
Average R² Test: 0.8455834415099956
Radius = 10.6087 + 1.2426*Face + 0.6941*NeighRadius + -0.0000*NeighFace
Radius = 10.6053 + 1.2523*Face + 0.7006*NeighRadius + -0.0000*NeighFace
Radius = 10.6009 + 1.2639*Face + 0.7024*NeighRadius + -0.0000*NeighFace
Radius = 10.6015 + 1.2763*Face + 0.7055*NeighRadius + -0.0000*NeighFace
Radius = 10.6009 + 1.2898*Face + 0.7062*NeighRadius + -0.0000*NeighFace
Radius = 10.6006 + 1.3037*Face + 0.7086*NeighRadius + -0.0000*NeighFace
Radius = 10.5998 + 1.3160*Face + 0.7091*NeighRadius + -0.0000*NeighFace
Radius = 10.6003 + 1.3298*Face + 0.7100*NeighRadius + -0.0000*NeighFace
Radius = 10.5980 + 1.3442*Face + 0.7101*NeighRadius + -0.0000*NeighFace
Radius = 10.5956 + 1.3584*Face + 0.7116*NeighRadius + -0.0000*NeighFace
Radius = 10.5970 + 1.3746*Face + 0.7113*NeighRadius + -0.0000*NeighFace
Radius = 10.5954 + 1.3889*Face + 0.7098*NeighRadius + -0.0

**R2 score = 0.85**